In [ ]:
import impulsegpt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from tqdm import tqdm
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
#import char_tokenizer

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS")
else:
    print("Using CPU")

In [ ]:
config = impulsegpt.Config()
config.ctx_len = 128
config.n_layers = 12
config.d_model = 768
config.n_heads = 12
config.vocab = 50000

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
collator = DataCollatorWithPadding(tokenizer, 'max_length', config.ctx_len, return_tensors='pt')

config.vocab = len(tokenizer.vocab)
print(f"Model vocab set to: {config.vocab}, Embedding size: {config.d_model * config.vocab}")


In [ ]:
ds = load_dataset("roneneldan/TinyStories")

In [ ]:
ds = ds['train']
ds = ds.shard(num_shards=100, index=0)

In [ ]:
print(f"Length before filter: {len(ds)}")
ds = ds.map(lambda t: tokenizer(t['text'], 
                                truncation=True, 
                                max_length=config.ctx_len,
                                return_overflowing_tokens=False,
                                return_length=True), batched=True)
ds = ds.remove_columns(['text','token_type_ids','attention_mask', 'length'])
ds = ds.filter(lambda t: len(t['input_ids']) > 2)
ds = ds.with_format('torch')

print(ds[0])
print(f"Train data length: {len(ds)}")
#print(f"Validation data length: {len(ds['validation'])}")

In [ ]:
dataloader = DataLoader(dataset=ds, collate_fn=collator, batch_size=8)

In [ ]:
model = impulsegpt.ImpulseGPT(config=config).to(device)
#model = torch.load('ckpt/ts-64-1.pt')
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005)
summary(model)

In [ ]:
def train(dataloader, model, loss_fn, optimizer, epochs:int, logger:SummaryWriter):
    model.train()
    print(f"Start training for {epochs} epochs with {len(dataloader)} steps each.")
    for s in range(epochs):
        pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {s+1} of {epochs}")
        for batch, row in pbar:
            step_loss = 0
            num_rows = row['input_ids'].shape[1] - 1
            for t in range(num_rows):
                context = row['input_ids'][...,:t+1].to(device)
                y = row['input_ids'][...,t+1].to(device)
                y_hat = model(context)
                loss = loss_fn(y_hat, y)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                step_loss += loss.item()
            step_loss /= num_rows
            logger.add_scalar('Loss', step_loss, batch+1)
            pbar.set_postfix({'Loss':step_loss})
            
    logger.close()


In [ ]:
writer = SummaryWriter()
train(dataloader, model, loss_fn, optimizer, epochs=1, logger=writer)

In [ ]:
torch.save(model, "ckpt/ts-64-1.pt")

In [ ]:
start_x = torch.tensor(tokenizer.encode('')).unsqueeze(dim=0).to(device=device)
print(start_x)
start_ids = torch.tensor([[101, 1377, 3221]]).to(device)
max_length = 32
y = model.generate(start_ids, max_length=max_length, top_k=64, temp=0.75)
print(y)
txt = tokenizer.decode(y[0].tolist(), skip_special_tokens=True)
print(y.shape)
print(txt)

In [ ]:
y = model(start_ids)
prob = nn.functional.softmax(y, dim=-1).cpu().detach().squeeze()
token_max = torch.argmax(prob)
print(token_max)
plt.plot(prob)
tokenizer.decode([token_max.tolist()])